In [1]:
%pwd

'/home/filip/Documents/cancer-classification/research'

In [2]:
import os

os.chdir("../")

In [3]:
%pwd

'/home/filip/Documents/cancer-classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import zipfile
import gdown
from cnnClassifier import logger
# from cnnClassifier.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config

    def download_data(self):
        """Downloads the dataset as a zip file from Google Drive."""
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file

            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}.")

            file_id = dataset_url.split("/")[-2]
            
            if os.path.exists(zip_download_dir):
                logger.info("Data already downloaded. Download stopped.")
            else:
                prefix = "https://drive.google.com/uc?/export=download&id="
                gdown.download(url=prefix + file_id, output=zip_download_dir)

                logger.info(
                    f"Downloaded data from {dataset_url} into file {zip_download_dir}"
                )
            
        except Exception as e:
            raise e

    def extract_zip_file(self):
        """Extracs the zip file into the data directory."""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(path=unzip_path)

In [15]:
try:
    config_manager = ConfigurationManager()#config_filepath=Path("../config/config.yaml"), params_filepath=Path("../params.yaml"))
    data_ingestion = DataIngestion(config_manager.get_data_ingestion_config())

    # NOTE: Downaload is skipped as the dataset is downloaded from Kaggle.
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()

    # I don't know why the damn create_directories won't work; creates directory then says that there is a problem with the isinstance func inside ensure lib???
    # error in question when using ensure_annotations: TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union
except Exception as e:
    raise e

[2024-05-09 22:55:30,298: INFO: common: YAML file: config/config.yaml loaded successfully!]
[2024-05-09 22:55:30,299: INFO: common: YAML file: params.yaml loaded successfully!]
[2024-05-09 22:55:30,300: INFO: common: Created directory at: artifacts]
[2024-05-09 22:55:30,300: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-05-09 22:55:30,300: INFO: 1760217869: Downloading data from https://drive.google.com/file/d/1TDOWgd3n580eMn6nHfdR2FSIxRURHEk_/view?usp=sharing into file artifacts/data_ingestion/data.zip.]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1TDOWgd3n580eMn6nHfdR2FSIxRURHEk_
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1TDOWgd3n580eMn6nHfdR2FSIxRURHEk_&confirm=t&uuid=d901eae0-eefd-4f0d-9140-fdd1d1f85166
To: /home/filip/Documents/cancer-classification/artifacts/data_ingestion/data.zip
100%|██████████| 124M/124M [00:06<00:00, 17.8MB/s] 

[2024-05-09 22:55:38,940: INFO: 1760217869: Downloaded data from https://drive.google.com/file/d/1TDOWgd3n580eMn6nHfdR2FSIxRURHEk_/view?usp=sharing into file artifacts/data_ingestion/data.zip]


artifacts/data_ingestion artifacts/data_ingestion/data.zip


In [13]:
# NOTE: (In)Sanity Check
from ensure import ensure_annotations

@ensure_annotations
def test(x: list):
    return

test(['abc'])